# Data Cleaning

### Configuración espacio de trabajo

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from repo_funciones import *


In [ ]:
#%pip install openpyxl

## Archivo 1: Censo de hogares y viviendas 2020

In [ ]:
censo=pd.read_excel('../DATA/censo_viv_CDMX.xlsx', sheet_name=10, skiprows=4, header=1) 
